In [30]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import os

import cv2

!rm *.jpg
!rm *.txt
!rm *.JPG


#Upload images to google colab
print("Upload Training Images...")
train_images = files.upload()
print("Upload Training Labels...")
train_labels = files.upload()
print("Upload Validation Images...")
validation_images = files.upload()
print("Upload Validation Labels...")
validation_labels = files.upload()
print("Upload Testing Images...")
test_images = files.upload()
print("Upload Testing Labels...")
test_labels = files.upload()


##Preprocessing of Images

#create lists of files for training, validation, and test images
train_images_list = list(train_images.keys())
validation_images_list = list(validation_images.keys())
test_images_list = list(test_images.keys())

#create lists of files for training, validation, and test labels
train_labels_list = list(train_labels.keys())
validation_labels_list = list(validation_labels.keys())
test_labels_list = list(test_labels.keys())

#get pixel array of training dataset
train_images_array = []
for image in train_images_list:
  image_pixel_array = cv2.imread(image)
  train_images_array.append(image_pixel_array)
#convert array to numpy array
train_images_nparray = np.array(train_images_array)

#get pixel array of validation dataset
validation_images_array = []
for image in validation_images_list:
  image_pixel_array = cv2.imread(image)
  validation_images_array.append(image_pixel_array)
#convert array to numpy array
validation_images_nparray = np.array(validation_images_array)

#get pixel array of test dataset
test_images_array = []
for image in test_images_list:
  image_pixel_array = cv2.imread(image)
  test_images_array.append(image_pixel_array)
#convert array to numpy array
test_images_nparray = np.array(test_images_array)

#make all pixel values between 0 and 1
train_images_nparray = train_images_nparray / 255.0
validation_images_nparray = validation_images_nparray / 255.0
test_images_nparray = test_images_nparray / 255.0

#Save all the labels in list form
train_class_list = []
for label_file in train_labels_list:
  label_file_open = open(label_file, "r")
  file_contents = label_file_open.read()
  object_class = file_contents.split('\t', 1)[0]
  if object_class == 'Coin':
    train_class_list.append(0)
  elif object_class == 'Cash':
    train_class_list.append(1)
train_class_nparray = np.asarray(train_class_list)

validation_class_list = []
for label_file in validation_labels_list:
  label_file_open = open(label_file, "r")
  file_contents = label_file_open.read()
  object_class = file_contents.split('\t', 1)[0]
  if object_class == 'Coin':
    validation_class_list.append(0)
  elif object_class == 'Cash':
    validation_class_list.append(1)
validation_class_nparray = np.asarray(validation_class_list)

test_class_list = []
for label_file in test_labels_list:
  label_file_open = open(label_file, "r")
  file_contents = label_file_open.read()
  object_class = file_contents.split('\t', 1)[0]
  if object_class == 'Coin':
    test_class_list.append(0)
  elif object_class == 'Cash':
    test_class_list.append(1)
test_class_nparray = np.asarray(test_class_list)

##Build Model
model = keras.Sequential([
        keras.layers.Conv2D(64, (6,6), activation='relu', input_shape=(80,80,3), data_format='channels_last'),
        keras.layers.Conv2D(32, (6,6), activation='relu', data_format='channels_last'),
        keras.layers.Flatten(), 
        keras.layers.Dense(2, activation='softmax')
        ])  

##Compile Model
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

##Train Model
model.fit(train_images_nparray, 
          train_class_nparray, 
          validation_data=(validation_images_nparray, validation_class_nparray),
          epochs=100,
          shuffle=True)

##Make Predictions
predictions = model.predict(test_images_nparray)
print(predictions)



rm: cannot remove '*.JPG': No such file or directory
Upload Training Images...


Saving 0a3af8dc-0000000.jpg to 0a3af8dc-0000000.jpg
Saving 0b242352-0000000.jpg to 0b242352-0000000.jpg
Saving 0cc711ed-0000000.jpg to 0cc711ed-0000000.jpg
Saving 0e2ce131-0000000.jpg to 0e2ce131-0000000.jpg
Saving 0f043e22-0000000.jpg to 0f043e22-0000000.jpg
Saving 1a1cc2f4-0000000.jpg to 1a1cc2f4-0000000.jpg
Saving 1a60e66c-0000000.jpg to 1a60e66c-0000000.jpg
Saving 1a131ff9-0000000.jpg to 1a131ff9-0000000.jpg
Saving 1aa0e8e8-0000000.jpg to 1aa0e8e8-0000000.jpg
Saving 1f8f810a-0000000.jpg to 1f8f810a-0000000.jpg
Saving 1f09712b-0000000.jpg to 1f09712b-0000000.jpg
Saving 02ca8bb6-0000000.jpg to 02ca8bb6-0000000.jpg
Saving 2b355e81-0000000.jpg to 2b355e81-0000000.jpg
Saving 2ccfca40-0000000.jpg to 2ccfca40-0000000.jpg
Saving 2dcc8f28-0000000.jpg to 2dcc8f28-0000000.jpg
Saving 3c8cbcf3-0000000.jpg to 3c8cbcf3-0000000.jpg
Saving 3f227e68-0000000.jpg to 3f227e68-0000000.jpg
Saving 4ac50ef0-0000000.jpg to 4ac50ef0-0000000.jpg
Saving 4bd86a87-0000000.jpg to 4bd86a87-0000000.jpg
Saving 4e456

Saving 0a3af8dc-0000000.txt to 0a3af8dc-0000000.txt
Saving 0b242352-0000000.txt to 0b242352-0000000.txt
Saving 0cc711ed-0000000.txt to 0cc711ed-0000000.txt
Saving 0e2ce131-0000000.txt to 0e2ce131-0000000.txt
Saving 0f043e22-0000000.txt to 0f043e22-0000000.txt
Saving 1a1cc2f4-0000000.txt to 1a1cc2f4-0000000.txt
Saving 1a60e66c-0000000.txt to 1a60e66c-0000000.txt
Saving 1a131ff9-0000000.txt to 1a131ff9-0000000.txt
Saving 1aa0e8e8-0000000.txt to 1aa0e8e8-0000000.txt
Saving 1f8f810a-0000000.txt to 1f8f810a-0000000.txt
Saving 1f09712b-0000000.txt to 1f09712b-0000000.txt
Saving 02ca8bb6-0000000.txt to 02ca8bb6-0000000.txt
Saving 2b355e81-0000000.txt to 2b355e81-0000000.txt
Saving 2ccfca40-0000000.txt to 2ccfca40-0000000.txt
Saving 2dcc8f28-0000000.txt to 2dcc8f28-0000000.txt
Saving 3c8cbcf3-0000000.txt to 3c8cbcf3-0000000.txt
Saving 3f227e68-0000000.txt to 3f227e68-0000000.txt
Saving 4ac50ef0-0000000.txt to 4ac50ef0-0000000.txt
Saving 4bd86a87-0000000.txt to 4bd86a87-0000000.txt
Saving 4e456

Saving 0e31fba2-0000000.jpg to 0e31fba2-0000000.jpg
Saving 0fe6b267-0000000.jpg to 0fe6b267-0000000.jpg
Saving 3a965ada-0000000.jpg to 3a965ada-0000000.jpg
Saving 3fba6594-0000000.jpg to 3fba6594-0000000.jpg
Saving 4cc7a96e-0000000.jpg to 4cc7a96e-0000000.jpg
Saving 8bc484d5-0000000.jpg to 8bc484d5-0000000.jpg
Saving 9e9dc853-0000000.jpg to 9e9dc853-0000000.jpg
Saving 12b8fa0d-0000000.jpg to 12b8fa0d-0000000.jpg
Saving 59d42d22-0000000.jpg to 59d42d22-0000000.jpg
Saving 60c8bef4-0000000.jpg to 60c8bef4-0000000.jpg
Saving 0437db4f-0000000.jpg to 0437db4f-0000000.jpg
Saving 5537a003-0000000.jpg to 5537a003-0000000.jpg
Saving 6602beef-0000000.jpg to 6602beef-0000000.jpg
Saving 7482ef10-0000000.jpg to 7482ef10-0000000.jpg
Saving 429130c9-0000000.jpg to 429130c9-0000000.jpg
Saving 990265da-0000000.jpg to 990265da-0000000.jpg
Saving 4434096b-0000000.jpg to 4434096b-0000000.jpg
Saving c9a1b7df-0000000.jpg to c9a1b7df-0000000.jpg
Saving ce487bae-0000000.jpg to ce487bae-0000000.jpg
Saving d08c3

Saving 0e31fba2-0000000.txt to 0e31fba2-0000000.txt
Saving 0fe6b267-0000000.txt to 0fe6b267-0000000.txt
Saving 3a965ada-0000000.txt to 3a965ada-0000000.txt
Saving 3fba6594-0000000.txt to 3fba6594-0000000.txt
Saving 4cc7a96e-0000000.txt to 4cc7a96e-0000000.txt
Saving 8bc484d5-0000000.txt to 8bc484d5-0000000.txt
Saving 9e9dc853-0000000.txt to 9e9dc853-0000000.txt
Saving 12b8fa0d-0000000.txt to 12b8fa0d-0000000.txt
Saving 59d42d22-0000000.txt to 59d42d22-0000000.txt
Saving 60c8bef4-0000000.txt to 60c8bef4-0000000.txt
Saving 0437db4f-0000000.txt to 0437db4f-0000000.txt
Saving 5537a003-0000000.txt to 5537a003-0000000.txt
Saving 6602beef-0000000.txt to 6602beef-0000000.txt
Saving 7482ef10-0000000.txt to 7482ef10-0000000.txt
Saving 429130c9-0000000.txt to 429130c9-0000000.txt
Saving 990265da-0000000.txt to 990265da-0000000.txt
Saving 4434096b-0000000.txt to 4434096b-0000000.txt
Saving c9a1b7df-0000000.txt to c9a1b7df-0000000.txt
Saving ce487bae-0000000.txt to ce487bae-0000000.txt
Saving d08c3

Saving 0ac05f95-0000000.jpg to 0ac05f95-0000000.jpg
Saving 0d649e4f-0000000.jpg to 0d649e4f-0000000.jpg
Saving 2ae039e7-0000000.jpg to 2ae039e7-0000000.jpg
Saving 5f61920c-0000000.jpg to 5f61920c-0000000.jpg
Saving 17d7d90d-0000000.jpg to 17d7d90d-0000000.jpg
Saving 33f3500b-0000000.jpg to 33f3500b-0000000.jpg
Saving 44edf722-0000000.jpg to 44edf722-0000000.jpg
Saving 70fddc43-0000000.jpg to 70fddc43-0000000.jpg
Saving 84fcb308-0000000.jpg to 84fcb308-0000000.jpg
Saving 98f87f97-0000000.jpg to 98f87f97-0000000.jpg
Saving 635d22d2-0000000.jpg to 635d22d2-0000000.jpg
Saving 65094c3c-0000000.jpg to 65094c3c-0000000.jpg
Saving 68522c64-0000000.jpg to 68522c64-0000000.jpg
Saving 12608024-0000000.jpg to 12608024-0000000.jpg
Saving b1c475ec-0000000.jpg to b1c475ec-0000000.jpg
Saving b49b29ec-0000000.jpg to b49b29ec-0000000.jpg
Saving b75287b1-0000000.jpg to b75287b1-0000000.jpg
Saving bac545d7-0000000.jpg to bac545d7-0000000.jpg
Saving cd960206-0000000.jpg to cd960206-0000000.jpg
Saving cf8a3

Saving 0ac05f95-0000000.txt to 0ac05f95-0000000.txt
Saving 0d649e4f-0000000.txt to 0d649e4f-0000000.txt
Saving 2ae039e7-0000000.txt to 2ae039e7-0000000.txt
Saving 5f61920c-0000000.txt to 5f61920c-0000000.txt
Saving 17d7d90d-0000000.txt to 17d7d90d-0000000.txt
Saving 33f3500b-0000000.txt to 33f3500b-0000000.txt
Saving 44edf722-0000000.txt to 44edf722-0000000.txt
Saving 70fddc43-0000000.txt to 70fddc43-0000000.txt
Saving 84fcb308-0000000.txt to 84fcb308-0000000.txt
Saving 98f87f97-0000000.txt to 98f87f97-0000000.txt
Saving 635d22d2-0000000.txt to 635d22d2-0000000.txt
Saving 65094c3c-0000000.txt to 65094c3c-0000000.txt
Saving 68522c64-0000000.txt to 68522c64-0000000.txt
Saving 12608024-0000000.txt to 12608024-0000000.txt
Saving b1c475ec-0000000.txt to b1c475ec-0000000.txt
Saving b49b29ec-0000000.txt to b49b29ec-0000000.txt
Saving b75287b1-0000000.txt to b75287b1-0000000.txt
Saving bac545d7-0000000.txt to bac545d7-0000000.txt
Saving cd960206-0000000.txt to cd960206-0000000.txt
Saving cf8a3